# **COCO-YOLO for Object Detection**

This tutorial will cover how a velour dataset integration can be made to speed up your workflow.

An integration can be useful if your organization uses a standard format for data 
or for when you just use a certain dataset often. We will use the COCO Panoptic dataset 
in this notebook to show ...

In [4]:
import os
import zipfile
import tempfile
import json
import PIL.Image
import requests
import numpy as np
from tqdm import tqdm
from io import BytesIO
from copy import deepcopy
from pathlib import Path, PosixPath
from typing import Any, Dict, List, Tuple, Union
from collections import defaultdict

from velour import Client, Dataset, Model, Datum, Annotation, Prediction, Label
from velour.enums import TaskType, JobStatus
from velour.viz import create_combined_segmentation_mask

import integrations.coco_integration as coco
import integrations.yolo_integration as yolo

import ultralytics

In [5]:
# connect
client = Client("http://localhost:8000")

Successfully connected to host at http://localhost:8000/


Using `coco_integration` create a dataset from COCO Panoptic trainval2017.

In [6]:
velour_dataset = coco.create_dataset_from_coco_panoptic(client, limit=2, reset=True)

Downloading:   0%|          | 706k/821M [00:00<19:02, 753kB/s]    


KeyboardInterrupt: 

Create a `Model` using the standard Velour initializer.

In [ ]:
velour_model = Model(client, "yolov8n-seg", reset=True)

Create predictions using YOLOv8 and upload these to Velour using `Model.add_prediction`

In [ ]:
inference_engine = ultralytics.YOLO(f"{velour_model.name}.pt")

for datum in tqdm(velour_dataset.get_datums()):

    image = coco.download_image(datum)

    results = inference_engine(image, verbose=False)

    # convert YOLO result into Velour Prediction
    prediction : Prediction = yolo.parse_object_detection(
        results,            # raw inference
        datum=datum,        # velour datum
        label_key='name',   # label_key override
    )

    # add prediction to the model
    velour_model.add_prediction(prediction)

The `coco_integration` handles dataset finalization internally. Since we only defined annotation parsers for YOLO we will have to finalize our inferences manually.

In [ ]:
velour_model.finalize_inferences(velour_dataset)

# **Exploring the Dataset**

In [ ]:
groundtruth_139 = velour_dataset.get_groundtruth('139')
groundtruth_139.datum.uid

In [ ]:
coco.download_image(groundtruth_139.datum)

In [ ]:
instance_mask, instance_legend = create_combined_segmentation_mask(
    [groundtruth_139], 
    label_key="name",
    task_type=TaskType.DETECTION,
)

In [ ]:
instance_mask

In [ ]:
for k, v in instance_legend.items():
    print(k)
    display(v)

# **Creating an Evaluation**

In [ ]:
eval1 = velour_model.evaluate_detection(velour_dataset)
eval1.wait_for_completion()

In [ ]:
eval1.results().dataframe

### **Evaluations with Filters**

In [ ]:
eval2 = velour_model.evaluate_detection(
    velour_dataset,
    filters=[
        Label.key == "name",
    ]
)
eval2.wait_for_completion()

In [ ]:
eval2.results().dataframe

In [ ]:
eval3 = velour_model.evaluate_detection(
    velour_dataset,
    filters=[
        Label.key == "name",
        Annotation.geometric_area <= 2000
    ]
)
eval3.wait_for_completion()

In [ ]:
eval3.results().dataframe